# Analyse des Einflusses von Battery2 auf Battery1

In [ ]:
### Importieren der verwendeten Bibilotheken
import os
import glob

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

### Einlesen der benötigten Daten

In [ ]:
pathToPickle_battery2_state = os.path.join("..", "..", "data", "TUDA_battery_state", "TUDA_battery_states_battery2_state.pickle")

# Einlesen der Daten
df_bat2Ana = pd.read_pickle(pathToPickle_battery2_state)

# Den "wagon_IDs" einen "wagon_type" mittels Mapping zuordnen
pathToMappingPickle = os.path.join("..", "..", "data", "mappingDf.pickle")
mappingdf = pd.read_pickle(pathToMappingPickle)
df_bat2Ana = pd.merge(df_bat2Ana, mappingdf, on = "wagon_ID")

# provider ist doppelt und wird gedroppt
df_bat2Ana.drop("provider_y", axis=1, inplace=True)
df_bat2Ana.rename(columns={'provider_x': 'provider'}, inplace=True)

### Aufteilen der Wagen mit Battery2 nach Wagontypes

In [ ]:
# Liste mit allen wagon_types wird erstellt
list_bat2_wagonTypes = np.sort(df_bat2Ana["wagon_type"].drop_duplicates().to_numpy())
list_bat2_wagonTypes

In [ ]:
# Liste mit einem DataFrame für jeden 'wagon_type' wird erstellt
df_bat2_state_list = []
for wagon_type in list_bat2_wagonTypes:
    df_bat2_state_list.append(df_bat2Ana.loc[df_bat2Ana['wagon_type'] == wagon_type])
df_bat2_state_list[4].head()
    

In [ ]:
def CalculateBatteryStateStatistics(dataFrame, BatteryTypeStr = 'battery_state'):
    '''
    Funktion, die sowohl für Battery als auch Battery2 Statistiken
    für einen bestinnen Zeitraum erstellen kann
    input:
        dataFrame
        'battery_state' / 'battery2_state' --> Auswahl des Batteriezustands; Standard: 'battery_state'
    '''
    #bins erstellen, in die die Daten eingeteilt und anschließend gemittelt werden sollen
    binSize = 1 # hier kann die Bingröße verändert werden; sinnvolle Werte sind binSize = 1 für einen Tag und binSize = 7 für eine Woche
    binsSeconds = np.arange(0, 45*7*24*60*60/binSize, binSize*24*60*60)
    binsSecondsLabels = [i for i, bin in enumerate(binsSeconds)]
    binsSeconds = binsSeconds.tolist()
    binsSecondsLabels.pop() # ein Wert muss entfernt werden, um gleich viele Labels wie bins zu definieren!
    
    # Daten mittels der timestamps in bins einteilen (jeweils immer genau einen bin zuweisen)
    dataFrame['bin'] = pd.cut(dataFrame['timestamp_measure_battery'], bins = binsSeconds, labels = binsSecondsLabels)
    
    batteryStateMeans = []
    for binNumber in binsSecondsLabels:
        # prüft für jede Reihe ob der bin der entsprechenden binnumber die gerade ausgewählt ist entspricht
        dfFiltered = dataFrame.loc[dataFrame['bin'] == binNumber] 
     
        batteryStateMeans.append([binNumber, dfFiltered[BatteryTypeStr].mean(), dfFiltered[BatteryTypeStr].std(), len(dfFiltered)])
    #DataFrame aus den Werten zusammensetzen
    batteryStateMeansDf = pd.DataFrame(batteryStateMeans, columns = ['start_day_bin', BatteryTypeStr + '_mean',BatteryTypeStr + '_std', 'binlength'])
    
    return batteryStateMeansDf

### Statistische Analyse für die verschiedenen 'wagon_types'

In [ ]:
bat2statistics = []

for dfBattery2State in df_bat2_state_list:
    bat2statistics.append(CalculateBatteryStateStatistics(df_bat2Ana, 'battery2_state'))
bat2statistics[4].head()

___

# Battery2 Analyse

In [ ]:
# Erstellung der Battery2 df
battery2_state_mean_df = CalculateBatteryStateStatistics(df_bat2Ana, 'battery2_state')
battery2_state_mean_df

In [ ]:
#sorted(pd.unique(df_battery2_state['battery2_state']))
# Es liegen im Rohdatensatz alle Werte von 0-100 % vor (natürliche Zahlen)

In [ ]:
#sorted(pd.unique(battery2_state_mean_df['battery2_state_mean']))
# Im bearbeiteten Datensatz liegen von 11 bis 99 reelle Zahlen vor

### Statistische Analyse des gesamten DataFrames von battery2 (ohne Einteilung / Zuweisung in verschiedene Kategorien)

In [ ]:
# Bins herausnehmen, an denen zu wenige Daten vorliegen
battery2_state_mean_df.drop(battery2_state_mean_df.loc[battery2_state_mean_df['binlength']<=3000].index, inplace=True)


In [ ]:
fig, ax = plt.subplots(ncols = 2, figsize=(30,7))
ax[0].plot(battery2_state_mean_df['start_day_bin'], battery2_state_mean_df['battery2_state_mean'], marker='o')
ax[1].scatter(battery2_state_mean_df['start_day_bin'], battery2_state_mean_df['battery2_state_mean'], c = battery2_state_mean_df['binlength'], marker='o', cmap = 'jet')


### Battery2 State List erstellen

In [ ]:
# Daten mittels der timestamps in bins einteilen (jeweils immer 1 Tag einen bin zuweisen)
df_battery_state['bin'] = pd.cut(df_battery_state['timestamp_measure_battery'], bins = bins_seconds, labels = bins_seconds_labels)

In [ ]:
# Liste mit allen wagon_types wird erstellt
wagon_type_list2 = np.sort(df_battery_state["wagon_type"].drop_duplicates().to_numpy())

In [ ]:
# Liste mit einem DataFrame für jeden 'wagon_type' wird erstellt
df_battery2_state_list = []
for wagon_type in wagon_type_list2:
    df_battery2_state_wagon_type = df_battery_state.loc[df_battery_state['wagon_type'] == wagon_type]
    df_battery2_state_list.append(df_battery2_state_wagon_type)
    

### Statistische Analyse für die verschiedenen 'wagon_types'

In [ ]:
battery2StatisticsWagonTypesList = []

for dfBattery2State in df_bat2_state_list:
    battery2StatisticsWagonTypesList.append(CalculateBatteryStateStatistics(dfBattery2State, 'battery2_state'))


In [ ]:
battery2StatisticsWagonTypesList[0].head()

In [ ]:
fig, ax = plt.subplots(ncols = 4, nrows=1, figsize=(7,3), sharex=True, sharey=True, constrained_layout=True)
ax = ax.flatten()

i = 0
for i, battery2StatisticWagonType in enumerate(battery2StatisticsWagonTypesList[0:4]):
    # Tage, an denen zu wenige Messwerte vorliegen vernachlässigen
    battery2StatisticWagonType.drop(battery2StatisticWagonType.loc[battery2StatisticWagonType['binlength']<=1000].index, inplace=True)

    ax[i].plot(battery2StatisticWagonType['start_day_bin'], battery2StatisticWagonType['battery2_state_mean'], marker='o')
    ax[i].plot(battery2StatisticWagonType['start_day_bin'], battery2StatisticWagonType['battery2_state_mean']+battery2StatisticWagonType['battery2_state_std'], color='grey')
    ax[i].plot(battery2StatisticWagonType['start_day_bin'], battery2StatisticWagonType['battery2_state_mean']-battery2StatisticWagonType['battery2_state_std'], color='grey')
    
    ax[i].fill_between(battery2StatisticWagonType['start_day_bin'], battery2StatisticWagonType['battery2_state_mean']+battery2StatisticWagonType['battery2_state_std'],
                       battery2StatisticWagonType['battery2_state_mean']-battery2StatisticWagonType['battery2_state_std'], color='lightgrey' , alpha=0.3)

    
    ax[i].set_title(f'Wagontyp {i+2}')
    ax[i].grid(True)
    ax[i].set_xlabel('Tag')
     
ax[0].set_ylabel(r'Batterieladezustand in $\%$')
# Speichern der Daten falls nötig, Pfad angeben
#fig.savefig(os.path.join('..', '..', 'data', 'battery2.pdf'))

___

In [ ]:
df_bat2Ana['Ausfall_Batterie'] = 0
df_bat2Ana['Ausfall_Batterie'].loc[df_bat2Ana.battery_state==0] = 1

In [ ]:
len_0 = len(df_bat2Ana.loc[df_bat2Ana.Ausfall_Batterie == 1])
len_n0 = len(df_bat2Ana.loc[df_bat2Ana.Ausfall_Batterie == 0])

print('Es sind '+str(len_0/(len_0+len_n0)*100)+' % der Battery_state Einträge von Provider 35 gleich 0.')
print('Absolut sind '+str(len_0)+' Einträge der batery_state gleich 0.')

In [ ]:
df_bat2Ana['Ausfall_Batterie'] = 0
df_bat2Ana['Ausfall_Batterie'].loc[df_bat2Ana.battery2_state==0] = 1

In [ ]:
len_0 = len(df_bat2Ana.loc[df_bat2Ana.Ausfall_Batterie == 1])
len_n0 = len(df_bat2Ana.loc[df_bat2Ana.Ausfall_Batterie == 0])

print('Es sind '+str(len_0/(len_0+len_n0)*100)+' % der Battery2_state Einträge von Provider 35 gleich 0.')
print('Absolut sind '+str(len_0)+' Einträge der batery2_state gleich 0.')

### Ermittlung des prozentualen Anteils von battery2 zu battery von Provider 35

In [ ]:
len_nan = len(df_bat2Ana[df_bat2Ana.battery2_state.isna()])
len_all = len(df_bat2Ana.loc[(df_bat2Ana['provider'] == 35)].wagon_ID.unique())
ratio_bat = (len_all-len_nan) / len_all
print(str(ratio_bat*100)+' % der von Provider 35 zur Verfügung gestellten Wagons besitzen eine zweite Batterie!')